### Import library yang digunakan

In [ ]:
import numpy as np
import pandas as pd
import random as rd
from random import randint
import matplotlib.pyplot as plt

Inisiasi Variabel

In [ ]:
solutions_per_pop = 40
num_generations = 80
crossover_rate = 0.8
mutation_rate = 0.4

### Barang-barang yang dibawa

#### Barang dalam koper

In [ ]:
item_number = np.arange(1,16)
item_name = np.array(["Pakaian", "Laptop", "Dokumen", "Alat Mandi", "Buku & Alat Tulis", "Kamera", "Sendal", "Makanan", "Selimut", "Boneka", "Sepatu", "Alat Rekreasi", "Minuman", "Alat Elektronik Lain", "Alat Sholat"])
weight = np.array([3, 3, 1, 2, 3, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1])
value = np.array([422, 380, 321, 376, 290, 350, 245, 320, 300, 120, 300, 190, 251, 280, 359])
knapsack_threshold = 16    #Maximum weight that the bag of thief can hold 
print('The list is as follows:')
print('Item No.   Weight   Value')
for i in range(item_number.shape[0]):
    print('{0}          {1}         {2}         {3}\n'.format(item_number[i], item_name[i], weight[i], value[i]))

#### Barang keseluruhan

In [ ]:
# item_number = np.arange(1,6)
# item_name = np.array(["Koper", "Gitar", "Peralatan Gaming", "Oleh-oleh", "Tripod", "Tas Jinjing"])
# weight = np.array([16, 2, 4, 6, 1, 3])
# value = np.array([680, 350, 420, 348, 320, 400])
# knapsack_threshold = 22    #Maximum weight that the bag of thief can hold 
# print('The list is as follows:')
# print('Item No.   Weight   Value')
# for i in range(item_number.shape[0]):
#     print('{0}          {1}         {2}         {3}\n'.format(item_number[i], item_name[i], weight[i], value[i]))

### Membuat populasi awal

In [ ]:
pop_size = (solutions_per_pop, item_number.shape[0])
print('Population size = {}'.format(pop_size))
initial_population = np.random.randint(2, size = pop_size)
initial_population = initial_population.astype(int)
print('Initial population: \n{}'.format(initial_population))

### Menghitung nilai fitness

In [ ]:
def cal_fitness(weight, value, population, threshold):
    fitness = np.empty(population.shape[0])
    for i in range(population.shape[0]):
        S1 = np.sum(population[i] * value)
        S2 = np.sum(population[i] * weight)
        if S2 <= threshold:
            fitness[i] = S1
        else :
            fitness[i] = 0
    return fitness.astype(int)


### Melakukan seleksi parent

In [ ]:
def selection(fitness, num_parents, population):
    fitness = list(fitness)
    parents = np.empty((num_parents, population.shape[1]))
    for i in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        parents[i,:] = population[max_fitness_idx[0][0], :]
        fitness[max_fitness_idx[0][0]] = -999999
    return parents

### Melakukan single point crossover

In [ ]:
def crossover(parents, num_offsprings):
    offsprings = np.empty((num_offsprings, parents.shape[1]))
    crossover_point = int(parents.shape[1]/2)
    i=0
    while (parents.shape[0] < num_offsprings):
        parent1_index = i%parents.shape[0]
        parent2_index = (i+1)%parents.shape[0]
        x = rd.random()
        if x > crossover_rate:
            continue
        parent1_index = i%parents.shape[0]
        parent2_index = (i+1)%parents.shape[0]
        offsprings[i,0:crossover_point] = parents[parent1_index,0:crossover_point]
        offsprings[i,crossover_point:] = parents[parent2_index,crossover_point:]
        i=+1
    return offsprings   

### Melakukan mutasi

In [ ]:

def mutation(offsprings):
    mutants = np.empty((offsprings.shape))
    for i in range(mutants.shape[0]):
        random_value = rd.random()
        mutants[i,:] = offsprings[i,:]
        if random_value > mutation_rate:
            continue
        int_random_value = randint(0,offsprings.shape[1]-1)    
        if mutants[i,int_random_value] == 0 :
            mutants[i,int_random_value] = 1
        else :
            mutants[i,int_random_value] = 0
    return mutants 

### Menghitung ulang 

In [ ]:
def optimize(weight, value, population, pop_size, num_generations, threshold):
    parameters, fitness_history = [], []
    num_parents = int(pop_size[0]/2)
    num_offsprings = pop_size[0] - num_parents 
    for i in range(num_generations):
        fitness = cal_fitness(weight, value, population, threshold)
        fitness_history.append(fitness)
        parents = selection(fitness, num_parents, population)
        offsprings = crossover(parents, num_offsprings)
        mutants = mutation(offsprings)
        population[0:parents.shape[0], :] = parents
        population[parents.shape[0]:, :] = mutants
        
    print('Last generation: \n{}\n'.format(population)) 
    fitness_last_gen = cal_fitness(weight, value, population, threshold)      
    print('Fitness of the last generation: \n{}\n'.format(fitness_last_gen))
    max_fitness = np.where(fitness_last_gen == np.max(fitness_last_gen))
    parameters.append(population[max_fitness[0][0],:])
    return parameters, fitness_history

### Hasil algoritma

In [ ]:
parameters, fitness_history = optimize(weight, value, initial_population, pop_size, num_generations, knapsack_threshold)
print('The optimized parameters for the given inputs are: \n{}'.format(parameters))
selected_items = item_number * parameters
print('\nSelected items that will maximize the knapsack without breaking it:')
for i in range(selected_items.shape[1]):
  if selected_items[0][i] != 0:
     print('{}\n'.format(selected_items[0][i]))

In [ ]:
fitness_history_mean = [np.mean(fitness) for fitness in fitness_history]
fitness_history_max = [np.max(fitness) for fitness in fitness_history]
plt.plot(list(range(num_generations)), fitness_history_mean, label = 'Mean Fitness')
plt.plot(list(range(num_generations)), fitness_history_max, label = 'Max Fitness')
plt.legend()
plt.title('Fitness through the generations')
plt.xlabel('Generations')
plt.ylabel('Fitness')
plt.show()
print(np.asarray(fitness_history).shape)